<div id="TOP" />
<a href="#END" > END of Notebook (filter) </a>

In [1]:
# -- CONFIG ------------------------------------
LAB_TAG=${PWD##*/} 
LAB_NAME="TFI_Azure_Lab8b. Importing Foreign Resources"
LAB_WEIGHT=82
LAB_MODE="Terraform"
# ----------------------------------------------

# NB=""; cd ~/nb/2021-Terraform/content/tf-labs/XXXX; set +x
DIE() { echo "[$PWD] Error: $*"; banner "ERROR"; exit 1; }
NBTOOL_RC=~/scripts/nbtool.rc; [ ! -f $NBTOOL_RC ] && DIE "No such script as $NBTOOL_RC"
echo "==== Reading nbtool functions from '$NBTOOL_RC' ===="; . $NBTOOL_RC
EXCL_FN_INIT_NOTEBOOK
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

==== Reading nbtool functions from '/home/student/scripts/nbtool.rc' ====
NB_DIR=/home/student/nb/2022-Terraform/content/tf-labs/8b.ImportingResources

==== Setting TF_DATA_DIR ====
TF_DATA_DIR=/home/student/dot.terraform

==== Setting AWS vars for student20 account ====
==== Terraform Lab weight[82] / tag[8b.ImportingResources] / name[Lab 8b. Importing Foreign Resources] ====
NB_DIR=/home/student/nb/2022-Terraform/content/tf-labs/8b.ImportingResources
NB=/home/student/nb/2022-Terraform/content/tf-labs/8b.ImportingResources/README.ipynb

==== Filtering current notebook [/home/student/nb/2022-Terraform/content/tf-labs/8b.ImportingResources/README.ipynb] ====
-- . ~/scripts/nbtool.rc

-- /usr/bin/python3 ~/scripts/nbtool.py -f ~/nb/2022-Terraform/content/tf-labs/8b.ImportingResources/README.ipynb
---- no VARS_SEEN ----
i/p nb has 0 variables
/home/student/nb/2022-Terraform/content/tf-labs/8b.ImportingResources/README.ipynb:
	#cells=41
cells to include[#31]=[[3, 4, 5, 6, 7, 8, 9, 10, 11, 

# --INCLUDE--SECTION--

## Background:

In this lab, we will see how we can import AWS resources which were not created by Terraform itself so that Terraform can be subsequently used to manage these resources.

**Note**: Be careful not to manage resource both *outside* and *inside* Terraform

We will look at importing resources using
- the ```terraform import``` command

Tasks.
- As before, create a new directory lab 8b, and create three files, resources.tf, main.tf and vars.tf.

## Tasks

## 1. Importing a resource using ```terraform import```

### First Create a new lab directory and move to that directory


In [23]:
mkdir -p ~/lab/lab8b

cd       ~/lab/lab8b

Use the command ```/usr/local/bin/get_instances.sh -ir``` to determine if there are already running instances.

If there are any you may wish to return to previous lab directories and perform ```terraform destroy``` to delete these resources.

The important thing is to know what instances if any are currently running

You should have no Terraform managed resources in the current workspace, verify this

#### First we will check if there are any instances running

Use the command ```/usr/local/bin/get_instances.sh -ir``` to determine if there are already running instances.

If there are any you may wish to return to previous lab directories and perform ```terraform destroy``` to delete these resources.

The important thing is to know what instances if any are currently running

You should have no Terraform managed resources in the current workspace, verify this

In [7]:
/usr/local/bin/get_instances.sh -ir


--  [count:0]-- All running instances -----------------


: 1

In [8]:
terraform show

No state.


#### Launch a new instance using the aws client

Launch a new aws_instance using the aws cli client using the following command:

In [9]:
aws ec2 run-instances --count 1  --tag-specifications \
    'ResourceType=instance,Tags=[{Key=aws-cli,Value=true}]' \
    --image-id ami-0e42deec9aa2c90ce --instance-type t2.micro

{
    "Groups": [],
    "Instances": [
        {
            "AmiLaunchIndex": 0,
            "ImageId": "ami-0e42deec9aa2c90ce",
            "InstanceId": "i-06817c3963d2716c9",
            "InstanceType": "t2.micro",
            "LaunchTime": "2022-01-14T13:11:49+00:00",
            "Monitoring": {
                "State": "disabled"
            },
            "Placement": {
                "AvailabilityZone": "us-west-1b",
                "GroupName": "",
                "Tenancy": "default"
            },
            "PrivateDnsName": "ip-172-31-0-7.us-west-1.compute.internal",
            "PrivateIpAddress": "172.31.0.7",
            "ProductCodes": [],
            "PublicDnsName": "",
            "State": {
                "Code": 0,
                "Name": "pending"
            },
            "StateTransitionReason": "",
            "SubnetId": "subnet-2c6ba676",
            "VpcId": "vpc-af4583c9",
            "Architecture": "x86_64",
            "BlockDeviceMappings": [],
   

**Note**: that we added a tag to identify this as a non-Terraform resource

**Note**: You may need to wait a minute for the VM to be visible ( in the running state )

then verify the running instances again using

In [10]:
/usr/local/bin/get_instances.sh -ir


--  [count:0]-- All running instances -----------------


: 1

In [11]:
/usr/local/bin/get_instances.sh -ir


--  [count:1]-- All running instances -----------------
{"id":"i-06817c3963d2716c9","image":"ami-0e42deec9aa2c90ce","state":"running","Tags":[{"Key":"aws-cli","Value":"true"}],"LaunchTime":"2022-01-14T13:11:49+00:00"}


: 1

Again, you should have no Terraform managed resources in the current workspace, verify this


#### Create a resource file for the running instance (not yet managed by Terraform)

We can create a simple terraform resource to be able to import this instance so it can be managed by terraform.

Create a new file import.tf with the **following form** with the **appropriate values** for the instance **you created** earlier:

```hcl
resource "aws_instance" "imported_item" {
    ami = "YOUR_IMAGE_ID"
    instance_type = "YOUR_TYPE"
}
```

In [47]:
#EXCLUDE

cat > import.tf <<EOF
resource "aws_instance" "imported_item" {
    ami           = "ami-0e42deec9aa2c90ce"
    instance_type = "t2.micro"
}
EOF

#. ~/scripts/nbtool.rc

SET_VAR AWS_ID $( /usr/local/bin/get_instances.sh -ir |
                grep -m 1 '"id":' |  sed -e 's/{"id":"//' -e 's/".*//' )
#echo $XX_AWS_ID
#terraform import aws_instance.imported_item $XX_AWS_ID

# TODO: FIX var setting ... do we need XX_VAR_AWS_ID in code cell ??

You can now import your resource - use your instance id from the above output, e.g. *"i-06817c3963d2716c9"* in the command below:

In [25]:
#EXCLUDE

# Let them think for themselves about the terraform init !!

terraform init


Initializing the backend...

Initializing provider plugins...
- Finding latest version of hashicorp/aws...
- Installing hashicorp/aws v3.72.0...
- Installed hashicorp/aws v3.72.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [26]:
terraform import aws_instance.imported_item

aws_instance.imported_item: Importing from ID "i-06817c3963d2716c9"...
aws_instance.imported_item: Import prepared!
  Prepared aws_instance for import
aws_instance.imported_item: Refreshing state... [id=i-06817c3963d2716c9]

Import successful!

The resources that were imported are shown above. These resources are now in
your Terraform state and will henceforth be managed by Terraform.



### Verify that terraform has imported the resource

Now verify that you can see the instance using 
- Looking at the newly created ```terraform.tfstate``` file
- Using the ```terraform show``` and ```terraform state list``` commands

In [27]:
#EXCLUDE

terraform show

terraform state list

# aws_instance.imported_item:
resource "aws_instance" "imported_item" {
    ami                                  = "ami-0e42deec9aa2c90ce"
    arn                                  = "arn:aws:ec2:us-west-1:604682923221:instance/i-06817c3963d2716c9"
    associate_public_ip_address          = true
    availability_zone                    = "us-west-1b"
    cpu_core_count                       = 1
    cpu_threads_per_core                 = 1
    disable_api_termination              = false
    ebs_optimized                        = false
    get_password_data                    = false
    hibernation                          = false
    id                                   = "i-06817c3963d2716c9"
    instance_initiated_shutdown_behavior = "stop"
    instance_state                       = "running"
    instance_type                        = "t2.micro"
    ipv6_address_count                   = 0
    ipv6_addresses                       = []
    monitoring                           = false


### Reflect on this

Think about what we just did, we

- created an *AWS EC2 instance* without using Terraform

- we then created a dummy *import.tf* partially describing the resource, containing just
  - the *ami_id*
  - the *instance_type*
  
- we then ran the ```terraform import``` command referencing the instance *id* of the EC2 instance

Once we performed these steps the resource has now been imported into the terraform state.

We can use this mechanism to start to manage infrastructure resources previously generated outside of *Terraform*.

**NOTE:** Note also that this is not very functional, there are a lot of manual steps to take to import a single resource - but 3rd-party tools exist ...

#### Now we can destroy the instance

You succeeded, you can now cleanup using ```terraform destroy```

Remove, or rename import.tf as import.tf.bak

In [28]:
#EXCLUDE
terraform destroy -auto-approve

aws_instance.imported_item: Refreshing state... [id=i-06817c3963d2716c9]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # aws_instance.imported_item will be destroyed
  - resource "aws_instance" "imported_item" {
      - ami                                  = "ami-0e42deec9aa2c90ce" -> null
      - arn                                  = "arn:aws:ec2:us-west-1:604682923221:instance/i-06817c3963d2716c9" -> null
      - associate_public_ip_address          = true -> null
      - availability_zone                    = "us-west-1b" -> null
      - cpu_core_count                       = 1 -> null
      - cpu_threads_per_core                 = 1 -> null
      - disable_api_termination              = false -> null
      - ebs_optimized                        = false -> null
      - get_password_data                    = false -> null
      -

### Verify that the terraform state is empty

Check the ```terraform.tfstate``` file and run ```terraform state list``` to verify that we have now destroyed the resource.

### Note that independently of Terraform the VM instance is nevertheless present in AWS but ```Terminating```

Note that the ```/usr/local/bin/get_instances.sh -ir``` command shows ```Running``` instances only ... the previously destroyed instance may still be present but in the ```Terminated``` or other state-*

You can verify none ```Running``` instances using either of the commands
- ```/usr/local/bin/get_instances.sh```
- or ```aws ec2 describe-instances```


## 2. [TODO] Importing resources using ```GoogleCloudPlatform/terraformer```


Terraformer is a tool from Google for importing resources from various providers.

See the list at https://github.com/GoogleCloudPlatform/terraformer#table-of-contents

The tool terraformer-aws has already been installed for you as ``` /usr/local/bin/terraformer-aws```

#### Create again an aws instance using the command

First verify that no instances are running:

In [29]:
/usr/local/bin/get_instances.sh -ir


--  [count:0]-- All running instances -----------------


: 1

In [30]:
aws ec2 run-instances --count 1 --tag-specifications \
    'ResourceType=instance,Tags=[{Key=aws-cli,Value=true}]' \
    --image-id ami-0e42deec9aa2c90ce --instance-type t2.micro

{
    "Groups": [],
    "Instances": [
        {
            "AmiLaunchIndex": 0,
            "ImageId": "ami-0e42deec9aa2c90ce",
            "InstanceId": "i-0d8f4825fb89d9333",
            "InstanceType": "t2.micro",
            "LaunchTime": "2022-01-14T13:53:59+00:00",
            "Monitoring": {
                "State": "disabled"
            },
            "Placement": {
                "AvailabilityZone": "us-west-1b",
                "GroupName": "",
                "Tenancy": "default"
            },
            "PrivateDnsName": "ip-172-31-0-38.us-west-1.compute.internal",
            "PrivateIpAddress": "172.31.0.38",
            "ProductCodes": [],
            "PublicDnsName": "",
            "State": {
                "Code": 0,
                "Name": "pending"
            },
            "StateTransitionReason": "",
            "SubnetId": "subnet-2c6ba676",
            "VpcId": "vpc-af4583c9",
            "Architecture": "x86_64",
            "BlockDeviceMappings": [],
 

### Import the new instance using ```terraformer-aws```

The ```terraformer-aws``` usage is documented at https://github.com/GoogleCloudPlatform/terraformer/blob/master/docs/aws.md

We will now use ```terraformer-aws``` to import all ```Running``` instances from the ```us-west-1``` region which we're using.

**Note:** It may also detect the previous ```Terminated``` instance as below, it will import only ```Running``` resources.

In [34]:
terraformer-aws import aws --resources=ec2_instance \
    --connect=true --regions=us-west-1 --profile ""

2022/01/14 14:01:01 aws importing region us-west-1
2022/01/14 14:01:03 aws importing... ec2_instance
2022/01/14 14:01:05 aws done importing ec2_instance
2022/01/14 14:01:05 Number of resources for service ec2_instance: 2
2022/01/14 14:01:05 Refreshing state... aws_instance.tfer--i-06817c3963d2716c9_
2022/01/14 14:01:05 Refreshing state... aws_instance.tfer--i-0d8f4825fb89d9333_
2022/01/14 14:01:05 ERROR: Read resource response is null for resource aws_instance.tfer--i-06817c3963d2716c9_
2022/01/14 14:01:11 ERROR: Unable to refresh resource tfer--i-06817c3963d2716c9_
2022/01/14 14:01:11 Filtered number of resources for service ec2_instance: 1
2022/01/14 14:01:11 aws Connecting.... 
2022/01/14 14:01:11 aws save ec2_instance
2022/01/14 14:01:11 aws save tfstate for ec2_instance


In [37]:
ll -tr generated/aws/ec2_instance

total 28
-rwxr-xr-x 1 student docker  124 Jan 14 14:01 provider.tf
-rwxr-xr-x 1 student docker  114 Jan 14 14:01 outputs.tf
-rwxr-xr-x 1 student docker 1446 Jan 14 14:01 instance.tf
drwxr-xr-x 3 student docker 4096 Jan 14 14:01 ..
-rwxr-xr-x 1 student docker 5413 Jan 14 14:01 terraform.tfstate
drwxr-xr-x 2 student docker 4096 Jan 14 14:01 .


In [38]:
cd generated/aws/ec2_instance

terraform state list

aws_instance.tfer--i-0d8f4825fb89d9333_


### Verify that the instance has been created

```Terraformer``` has created a subdirectory with the appropriate Terraform config files and a ```terraform.tfstate``` file.

This was a much nicer experience than using ```terraform import``` directly.

### Browse the documentation

This is quite a powerful tool, facilitating the importing of resources into ```Terraform```

- discovery of resources by type, by region

- importing of resources from many provider types, not just AWS.

Take a look at the documentation at https://github.com/GoogleCloudPlatform/terraformer, 


### Delete the instance has been created

You will need to be in the ```generated/aws/ec2_instance``` directory of course to perform the ```terraform destroy```


# **Stretch Goal**: Import other resource types


Try now to import other resource types, such as an S3 bucket, using *terraform import*' and/or '*terraforming*' tools

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg) --> <img src="../../../static/images/LOGO_v2_CROPPED.jpg" width="200" />

# --EXCLUDE--SECTION--

## **NOTE:** Save notebook first

<div id="END" /> <a href="#TOP" > TOP OF PAGE </a>  <!-- must appear after EXCLUDE pragma !! -->

In [50]:
NB_DIR=~/nb/2021-Terraform/content/tf-labs/8b.ImportingResources/
. ~/scripts/nbtool.rc
pwd
EXCL_FN_FILTER_NOTEBOOK $NB

/home/student/nb/2021-Terraform/content/tf-labs/8b.ImportingResources

==== Filtering current notebook ====
-- . ~/scripts/nbtool.rc

-- /usr/bin/python3 ~/scripts/nbtool.py -f README.ipynb
cells to include[#31]=[[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 21, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]]
cells to include[#31]=[[37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 24, 23, 21, 20, 18, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3]]
-rw-r--r-- 1 student docker 23655 Jan 14 14:25 ~/nb/2021-Terraform/content/tf-labs/8b.ImportingResources//README.ipynb.filtered.ipynb

Good: No VAR defs/usages in i/p file

Good: No VAR defs/usages in o/p file

-- jupyter-nbconvert --to markdown README.ipynb.filtered.ipynb
[NbConvertApp] Converting notebook README.ipynb.filtered.ipynb to markdown
[NbConvertApp] Writing 18292 bytes to README.ipynb.filtered.md

-rw-r--r--  1 student docker    70 Jan 14 14:20 .dir
-rw-r--r--  1 student docker 23655 Jan 14 14:25 .ipynb
-rw-r